We must detail that we are using a dictionary instead of a database which would be used in a production environment 

In [ ]:
from flask import Flask
from flask_restful import Api, Resource, reqparse
import numpy as np
import pickle as p
import sys

APP = Flask(__name__)
API = Api(APP)
sampleID = 0

loaded_model = p.load(open('RandomForest.pickle', 'rb'))
samples = {}

class Predict(Resource):
       
    #POST - Here we will classify a new sample
    @staticmethod
    def post():
        global sampleID
        json_ = request.get_json()
        d = json.loads(json_)
        
        
        dictionary = {}
  
        for i in range(len(d['data'])):
            name = d['index'][i]
            dictionary[name] = d['data'][i]
        
        query = pd.DataFrame(dictionary, index=[0])

        prediction = loaded_model.predict(query)

        out = {'Prediction': int(prediction[0])}
        
        #so now we should append this to the samples dictionary 
        sample = d['data'] + prediction[0]
        
        #append the sample to the dictionary
        samples[sampleID] = sample 
        sampleID += 1
        
       
        return (out, 200)
    
    #GET - This will return the name of the model we are using currently
    @staticmethod
    def get():
        
        #return the model name 
        #return 200
        return type(loaded_model).__name__, 200
    
    
class Retrain(Resource):

    #POST - Here will retrain the model with new data 
    @staticmethod
    def post():

        json_ = request.get_json()
        d = json.loads(json_)

        train_data = pd.DataFrame(data = d['data'], columns = d['columns'])
        y_train = train_data['CarInsurance']
        X_train = train_data.drop(labels='CarInsurance', axis=1)
        
        loaded_model.fit(X_train, y_train)

        score = loaded_model.score(X_train, y_train)
        print(score)
        
        #Model is only as good as random and so we should flag something 
        #perhaps this error message isnt the best but it is just to show the idea 
        if score <= 50:
            return "Request Completed (200) - Poor Model Performance"
        
        return 200
    
class Change_Model(Resource):

    #POST - Here will retrain the model with new data 
    @staticmethod
    def post():
        global loaded_model

        json_ = request.get_json()
        d = json.loads(json_)

        loaded_model = p.load(open(str(d['model']), 'rb'))
        
        #Do a quick check to see if the new model name is now correct 
        substring = str(d['model'])[:7]
        
        #Do some error checking (I know for production grade code better error detection will be required)
        if substring in type(loaded_model).__name__:
            return 200
            
        return "Error - Model did not update correctly"
    
class Database(Resource):

    #POST - Here will retrain the model with new data 
    @staticmethod
    def get():
        
        if len(samples) == 0:
            return "The database is empty"
        
        json_str = json.dumps({k: v.tolist() for k, v in samples.items()})

        return json_str
    
    
    #PUT - Here will update the database sample given with the new data
    @staticmethod
    def put():
        
        json_ = request.get_json()
        d = json.loads(json_)
        
        valuesList = list(d.values())
        index = valuesList[14]
        valuesList = valuesList[:14]
        
        print(valuesList)
            
        #Check if the index of the sample is in the database and if not throw an error
        if index not in samples:
            return "Error - The sample index supplied is not in the database"
        else:
            samples[index] = valuesList
            
        return 200
    
    #DELETE - This query will delete the specified record 
    @staticmethod
    def delete():
        
        json_ = request.get_json()
        index = json.loads(json_)
        
        print(index)
            
        #Check if the index of the sample is in the database and if not throw an error
        if index not in samples:
            return "Error - The sample index supplied is not in the database"
        else:
            del samples[index]
            
        if index in samples:
            return("Deleting this sample did not work")
            
        return 200


API.add_resource(Predict, '/predict')
API.add_resource(Retrain, '/retrain')
API.add_resource(Change_Model, '/change_model')
API.add_resource(Database, '/database')

if __name__ == '__main__':
    APP.run(debug=True, port='1080', use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:1080/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Jun/2021 19:48:33] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2021 19:48:36] "POST /retrain HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2021 19:48:36] "POST /change_model HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2021 19:48:36] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2021 19:48:36] "GET /database HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2021 19:48:36] "PUT /database HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2021 19:48:36] "DELETE /database HTTP/1.1" 200 -


0.8678125
[0.0945945946, 0.0909090909, 1.0, 0.6666666667, 0.0, 0.0454253612, 1.0, 1.0, 0.0, 0.3333333333, 0.0, 0.0, 0.0, 0.0058631922]
0


127.0.0.1 - - [06/Jun/2021 19:48:38] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2021 19:48:38] "POST /predict HTTP/1.1" 200 -
